## 基于MindNLP+MusicGen生成自己的个性化音乐

MusicGen是来自Meta AI的Jade Copet等人提出的基于单个语言模型（LM）的音乐生成模型，能够根据文本描述或音频提示生成高质量的音乐样本，相关研究成果参考论文《[Simple and Controllable Music Generation](https://arxiv.org/abs/2306.05284)》。

MusicGen模型基于Transformer结构，可以分解为三个不同的阶段:
1. 用户输入的文本描述作为输入传递给一个固定的文本编码器模型，以获得一系列隐形状态表示。
2. 训练MusicGen解码器来预测离散的隐形状态音频token。
3. 对这些音频token使用音频压缩模型（如EnCodec）进行解码，以恢复音频波形。

MusicGen直接使用谷歌的[t5-base](https://huggingface.co/t5-base)及其权重作为文本编码器模型，并使用[EnCodec 32kHz](https://huggingface.co/facebook/encodec_32khz)及其权重作为音频压缩模型。MusicGen解码器是一个语言模型架构，针对音乐生成任务从零开始进行训练。


MusicGen 模型的新颖之处在于音频代码的预测方式。传统上，每个码本都必须由一个单独的模型（即分层）或通过不断优化 Transformer 模型的输出（即上采样）进行预测。与传统方法不同，MusicGen采用单个stage的Transformer LM结合高效的token交织模式，取消了多层级的多个模型结构，例如分层或上采样，这使得MusicGen能够生成单声道和立体声的高质量音乐样本，同时提供更好的生成输出控制。MusicGen不仅能够生成符合文本描述的音乐，还能够通过旋律条件控制生成的音调结构。

<p align="center">
  <img src="https://github.com/sanchit-gandhi/codesnippets/blob/main/delay_pattern.png?raw=true" width="600"/>
</p>


**Figure 1:** MusicGen使用的码本延迟模式，来源于 [MusicGen paper](https://arxiv.org/abs/2306.05284).


## 下载模型

MusicGen提供了small、medium和big三种规格的[预训练权重文件](https://huggingface.co/models?search=facebook/musicgen-)，本次指南默认使用small规格的权重，生成的音频质量较低，但是生成的速度是最快的：

In [ ]:
%%capture captured_output
# 实验环境已经预装了mindspore==2.4.10，如需更换mindspore版本，可更改下面 MINDSPORE_VERSION 变量
!pip uninstall mindspore -y
%env MINDSPORE_VERSION=2.4.10
!pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/${MINDSPORE_VERSION}/MindSpore/unified/x86_64/mindspore-${MINDSPORE_VERSION}-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

In [ ]:
# 查看当前 mindspore 版本
!pip show mindspore

In [ ]:
# 安装 mindnlp 包
!pip install -i https://pypi.mirrors.ustc.edu.cn/simple mindnlp==0.2.4 jieba

In [3]:
from mindnlp.transformers import MusicgenForConditionalGeneration

model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.669 seconds.
Prefix dict has been built successfully.


## 生成音乐

MusicGen支持两种生成模式：贪心（greedy）和采样（sampling）。在实际执行过程中，采样模式得到的结果要显著优于贪心模式。因此我们默认启用采样模式，并且可以在调用`MusicgenForConditionalGeneration.generate`时设置`do_sample=True`来显式指定使用采样模式。

### 无提示生成

我们可以通过方法 `MusicgenForConditionalGeneration.get_unconditional_inputs` 获得网络的随机输入，然后使用 `.generate` 方法进行自回归生成，指定 `do_sample=True` 来启用采样模式:

In [4]:
unconditional_inputs = model.get_unconditional_inputs(num_samples=1)

audio_values = model.generate(**unconditional_inputs, do_sample=True, max_new_tokens=256)

音频输出是格式是: a Torch tensor of shape `(batch_size, num_channels, sequence_length)`。

使用第三方库`scipy`将输出的音频保存为`musicgen_out.wav` 文件。

In [5]:
import scipy

sampling_rate = model.config.audio_encoder.sampling_rate
scipy.io.wavfile.write("musicgen_out.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [6]:
# from IPython.display import Audio
# # 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
# Audio(audio_values[0].asnumpy(), rate=sampling_rate)

参数 `max_new_tokens` 指定要生成 `token` 数。根据经验，可以使用 `EnCodec` 模型的帧速率计算出生成的音频样本的长度（以秒为单位）：

In [7]:
# audio_length_in_s = 256 / model.config.audio_encoder.frame_rate

# audio_length_in_s

### 文本提示生成

首先基于文本提示，通过`AutoProcessor`对输入进行预处理。然后将预处理后的输入传递给 `.generate` 方法以生成文本条件音频样本。同样，我们通过设置“do_sample=True”来启用采样模式。

其中，`guidance_scale` 用于无分类器指导（CFG），设置条件对数之间的权重（从文本提示中预测）和无条件对数（从无条件或空文本中预测）。`guidance_scale`越高表示生成的模型与输入的文本更加紧密。通过设置`guidance_scale > 1`来启用 CFG。为获得最佳效果，使用`guidance_scale=3`（默认值）生成文本提示音频。

In [8]:
from mindnlp.transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    text=["80s pop track with bassy drums and synth", "90s rock song with loud guitars and heavy drums"],
    padding=True,
    return_tensors="ms",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

In [9]:
scipy.io.wavfile.write("musicgen_out_text.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [10]:
# from IPython.display import Audio
# # 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
# Audio(audio_values[0].asnumpy(), rate=sampling_rate)

### 音频提示生成

`AutoProcessor`同样可以对用于音频预测的音频提示进行预处理。在以下示例中，我们首先加载音频文件，然后进行预处理，并将输入给到网络模型来进行音频生成。最后，我们将生成出来的音频文件保存为` musicgen_out_audio.wav `

In [11]:
from datasets import load_dataset

dataset = load_dataset("sanchit-gandhi/gtzan", split="train", streaming=True)
sample = next(iter(dataset))["audio"]

# take the first half of the audio sample
sample["array"] = sample["array"][: len(sample["array"]) // 2]

In [12]:
# 使用音视频提示生成，耗时较久
processor = AutoProcessor.from_pretrained("facebook/musicgen-small")

inputs = processor(
    audio=sample["array"],
    sampling_rate=sample["sampling_rate"],
    text=["80s blues track with groovy saxophone"],
    padding=True,
    return_tensors="ms",
)

audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

In [13]:
scipy.io.wavfile.write("musicgen_out_audio.wav", rate=sampling_rate, data=audio_values[0, 0].asnumpy())

In [14]:
# from IPython.display import Audio
# # 要收听生成的音频样本，可以使用 Audio 在 notebook 进行播放
# Audio(audio_values[0].asnumpy(), rate=sampling_rate)